# 第9章: ベクトル空間法 (I)
[enwiki-20150112-400-r10-105752.txt.bz2](http://www.cl.ecei.tohoku.ac.jp/nlp100/data/enwiki-20150112-400-r10-105752.txt.bz2)は，2015年1月12日時点の英語のWikipedia記事のうち，約400語以上で構成される記事の中から，ランダムに1/10サンプリングした105,752記事のテキストをbzip2形式で圧縮したものである．このテキストをコーパスとして，単語の意味を表すベクトル（分散表現）を学習したい．第9章の前半では，コーパスから作成した単語文脈共起行列に主成分分析を適用し，単語ベクトルを学習する過程を，いくつかの処理に分けて実装する．第9章の後半では，学習で得られた単語ベクトル（300次元）を用い，単語の類似度計算やアナロジー（類推）を行う．

なお，問題83を素直に実装すると，大量（約7GB）の主記憶が必要になる． メモリが不足する場合は，処理を工夫するか，1/100サンプリングのコーパス[enwiki-20150112-400-r100-10576.txt.bz2](http://www.cl.ecei.tohoku.ac.jp/nlp100/data/enwiki-20150112-400-r100-10576.txt.bz2)を用いよ．

In [1]:
!ls

country.txt                        section9.ipynb
enwiki-20150112-400-r10-105752.txt token.text
enwiki-20150112-400-r100-10576.txt token.txt


## 80. コーパスの整形
文を単語列に変換する最も単純な方法は，空白文字で単語に区切ることである． ただ，この方法では文末のピリオドや括弧などの記号が単語に含まれてしまう． そこで，コーパスの各行のテキストを空白文字でトークンのリストに分割した後，各トークンに以下の処理を施し，単語から記号を除去せよ．

- トークンの先頭と末尾に出現する次の文字を削除: `.,!?;:()[]'"`
- 空文字列となったトークンは削除

以上の処理を適用した後，トークンをスペースで連結してファイルに保存せよ．

In [2]:
# import re
token = []
# pattern = r'(.,!?;:()[])?(?P<text>.*?)(.,!?;:()[])'
# pattern = r'''[.,!?;:()[]'"]?((?P<text>.*?)[.,!?;:()[]'"]|(P<text>.*?))'''
# prog = re.compile(pattern)
with open("enwiki-20150112-400-r100-10576.txt",'r') as f:
    for line in f.readlines():
        line = line.strip()
        line = line.split(' ')
        for l in line:
            if len(l) <= 0:
                continue
            if l[0] in list('''.,!?;:'"()[]'''):
                l = l[1:]
            if len(l) <= 0:
                continue
            if l[-1] in list('''.,!?;:'"()[]'''):
                l = l[:-1]
            token.append(l)

token = ' '.join(token)

In [3]:
with open("token.txt",'w') as f:
    f.write(token)

## 81. 複合語からなる国名への対処
英語では，複数の語の連接が意味を成すことがある．例えば，アメリカ合衆国は"United States"，イギリスは"United Kingdom"と表現されるが，"United"や"States"，"Kingdom"という単語だけでは，指し示している概念・実体が曖昧である．そこで，コーパス中に含まれる複合語を認識し，複合語を1語として扱うことで，複合語の意味を推定したい．しかしながら，複合語を正確に認定するのは大変むずかしいので，ここでは複合語からなる国名を認定したい．

インターネット上から国名リストを各自で入手し，80のコーパス中に出現する複合語の国名に関して，スペースをアンダーバーに置換せよ．例えば，"United States"は"United_States"，"Isle of Man"は"Isle_of_Man"になるはずである．

In [4]:
!curl https://www.benricho.org/translate/countrycode.html > country.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  107k    0  107k    0     0   143k      0 --:--:-- --:--:-- --:--:--  143k


In [5]:
import bs4
with open("country.txt",'r') as f:
    soup = bs4.BeautifulSoup(f.read())

In [6]:
country = []
for tbody in soup.find_all("tbody"):
    for tr in tbody.find_all("tr"):
        c = tr.find_all("td")[-2].text
        if 1 < len(c.split(' ')):
            c = c.split('\n')[0]
            country.append(c)

In [7]:
country

['United States',
 'American Samoa',
 'Virgin Islands, U.S.',
 'United Arab Emirates',
 'Antigua and Barbuda',
 'United Kingdom',
 'British Indian Ocean Territory',
 'Virgin Islands, British',
 'Iran, Islamic Republic of',
 'El Salvador',
 'Aland Islands',
 'United States Minor Outlying Islands',
 'Cape Verde',
 'North Macedonia, Republic of',
 'Northern Mariana Islands',
 'Cook Islands',
 'Christmas Island',
 'Cayman Islands',
 "Côte d'Ivoire",
 'Cocos (Keeling) Islands',
 'Costa Rica',
 'Congo, the Democratic Republic of the',
 'Saudi Arabia',
 'South Georgia and the South Sandwich Islands',
 'Sao Tome and Principe',
 'Saint Barthélemy',
 'Saint Pierre and Miquelon',
 'San Marino',
 'Saint Martin (French part)',
 'Sierra Leone',
 'Syrian Arab Republic',
 'Sint Maarten (Dutch part)',
 'Svalbard and Jan Mayen',
 'Sri Lanka',
 'Equatorial Guinea',
 'Saint Kitts and Nevis',
 'Saint Vincent and the Grenadines',
 'Saint Helena, Ascension and Tristan da Cunha',
 'Saint Lucia',
 'Solomon Isl

In [30]:
# import re
for pattern in country[:10]:
    token = token.replace(pattern,pattern.replace(" ",'_'))

with open("corpus.txt",'w') as f:
    f.write(token)

       0 11886138 71766399 corpus.txt


## 82. 文脈の抽出
81で作成したコーパス中に出現するすべての単語tに関して，単語tと文脈語cのペアをタブ区切り形式ですべて書き出せ．ただし，文脈語の定義は次の通りとする．

ある単語tの前後d単語を文脈語cとして抽出する（ただし，文脈語に単語tそのものは含まない）
単語tを選ぶ度に，文脈幅dは{1,2,3,4,5}の範囲でランダムに決める．

In [37]:
with open("corpus.txt",'r') as f:
    token = f.readlines()

In [9]:
import  random
    
def context(sentence):
    for i,t in enumerate(sentence):
        d = random.randint(1,5)
        c = [tmp[i+a] for a in range(1,d+1) if i+a < len(tmp)] +  [tmp[i-a] for a in range(1,d+1) if 0<=i-a]
        yield d,t,c
    
# for d,t,c in context(tmp):
#     print("{}\tt={}\tc=".format(d,t),end='')
#     for ct in c:
#         print("{}".format(ct),end='\t')
#     print()

## 83. 単語／文脈の頻度の計測
82の出力を利用し，以下の出現分布，および定数を求めよ．

- $f(t,c)$: 単語$t$と文脈語$c$の共起回数
- $f(t,∗)$: 単語$t$の出現回数
- $f(∗,c)$: 文脈語$c$の出現回数
- $N$: 単語と文脈語のペアの総出現回数

In [10]:
from collections import defaultdict

f_t_c = defaultdict(lambda: defaultdict(int))
f_t = defaultdict(int)
f_c = defaultdict(int)
N = 0
f_t_c_list = list()



# for i,t in enumerate(tmp):
#     f_t[t] += 1
#     d = random.randint(1,5)
#     c = [tmp[i+a] for a in range(1,d+1) if i+a < len(tmp)] +  [tmp[i-a] for a in range(1,d+1) if 0<=i-a]
#     for word in c:
#         f_c[word] += 1
#         f_t_c[t][word] +=1
#         N+=1

for _,t,c in context(tmp):
    f_t[t] += 1
    for word in c:
        f_c[word] += 1
        f_t_c[t][word] += 1
        N += 1
        if 10 == f_t_c[t][word]:
            f_t_c_list.append((t,word))



# 84. 単語文脈行列の作成
83の出力を利用し，単語文脈行列$X$を作成せよ．ただし，行列$X$の各要素$X_{tc}$は次のように定義する．

- $f(t,c)≥10$ならば，$Xtc=\rm{PPMI}(t,c)=max{logN×f(t,c)f(t,∗)×f(∗,c),0}$
- $f(t,c)<10$ならば，$X_{tc}=0$

ここで，PPMI(t,c)はPositive Pointwise Mutual Information（正の相互情報量）と呼ばれる統計量である．なお，行列Xの行数・列数は数百万オーダとなり，行列のすべての要素を主記憶上に載せることは無理なので注意すること．幸い，行列Xのほとんどの要素は0になるので，非0の要素だけを書き出せばよい．

In [11]:
word_set = set(tmp)
s_to_i = dict((s,i) for i,s in enumerate(word_set))
i_to_s = dict((i,s) for s,i in s_to_i.items())

In [14]:
from scipy.sparse import csr_matrix, lil_matrix

X = lil_matrix( (len(f_t),len(f_c)))
# X = csr_matrix( (len(f_t),len(f_c)))

In [19]:
import numpy as np
# for t in f_t_c.keys():
#     for c in f_t_c[t].keys():
#         if 10 <= f_t_c[t][c]:
#             X[s_to_i[t],s_to_i[c]] = max(0,np.log(float(N*f_t_c[t][c]*f_t[t]*f_c[c])))


for t,c in f_t_c_list:
    X[s_to_i[t],s_to_i[c]] = max(0,np.log(float(N*f_t_c[t][c]*f_t[t]*f_c[c])))

## 85. 主成分分析による次元圧縮
84で得られた単語文脈行列に対して，主成分分析を適用し，単語の意味ベクトルを300次元に圧縮せよ．


In [20]:
import numpy as np
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=300)
feature = svd.fit_transform(X)

In [21]:
print(feature.shape)

(438648, 300)


## 86. 単語ベクトルの表示
85で得た単語の意味ベクトルを読み込み，"United States"のベクトルを表示せよ．ただし，"United States"は内部的には"United_States"と表現されていることに注意せよ．

In [22]:
def vector(s):
    return feature[s_to_i[s]]

In [23]:
vector("United_States")

array([ 4.18407036e+02, -8.20843424e+01, -2.59343088e+02,  4.27962737e+01,
        1.08326217e+02, -3.75027426e+01, -1.28787476e+02, -1.32346426e+01,
       -1.94323768e+01, -2.80035185e+00,  1.58321515e+01,  1.05151025e+01,
       -1.80908124e+01,  8.50995346e+01,  1.04968388e+01, -5.12349526e+01,
        1.00098070e+01,  7.79193247e+00, -2.13391285e-01, -9.48318731e+01,
       -7.33784436e+01, -1.12933899e+01,  3.55734640e+01, -3.41134604e+01,
       -7.29979335e-01, -1.50716197e+01, -2.66369854e+01, -2.72981826e+01,
        4.29473600e+00,  9.32350876e-01, -1.56153846e+00,  7.02239444e+01,
       -5.56892843e+01, -1.79719895e+01,  1.68210657e+01, -1.94133996e+00,
       -4.90085353e+01,  1.84052842e+01,  3.72716544e+01, -7.02428677e+00,
        3.89019746e+00,  7.87880192e+00, -7.95369744e+00,  4.59310913e+01,
       -1.42935720e+01, -1.71287487e+01, -1.65649504e+01, -2.78420855e+01,
        2.10687049e+01,  3.81843882e+01, -7.37938037e+01, -2.27896029e+01,
        3.88810665e+01,  

## 87. 単語の類似度
85で得た単語の意味ベクトルを読み込み，"United States"と"U.S."のコサイン類似度を計算せよ．ただし，"U.S."は内部的に"U.S"と表現されていることに注意せよ．

In [55]:
epsilon = 1e-10
def c_similarity(a,b):
#     ret = np.sum(np.array(a)*np.array(b))/(np.linalg.norm(a,ord=2)*np.linalg.norm(b,ord=2))
#     print(np.seterr('raise'))
#     return ret
    return np.dot(a, b) / ((np.linalg.norm(a) * np.linalg.norm(b)) + epsilon )
    

In [56]:
a = vector("United_States")
b = vector("U.S.")
print(c_similarity(a,b))

0.1946370580032267


## 88. 類似度の高い単語10件
85で得た単語の意味ベクトルを読み込み，"England"とコサイン類似度が高い10語と，その類似度を出力せよ．

In [57]:
import heapq as hq
def top10(a):
    q = list()
    hq.heapify(q)
    for i in range(len(feature)):
        hq.heappush(q,[-1*c_similarity(a,feature[i]),i_to_s[i]])
    
    for i in range(10):
        tmp = hq.heappop(q)
        print(-1*tmp[0],tmp[1])

In [58]:

England = vector("England")
print(England[:2])
top10(England)

[ 310.56277611 -125.3156941 ]
0.9999999999999996 England
0.7767086335259875 Australia
0.7717645227326353 France
0.7709907112286484 Europe
0.7483358245892583 Germany
0.7309423368461656 Japan
0.7274289591522056 India
0.7239157723025128 America
0.6966131919434029 London
0.6926415479899415 once


 ## 89. 加法構成性によるアナロジー
85で得た単語の意味ベクトルを読み込み，vec("Spain") - vec("Madrid") + vec("Athens")を計算し，そのベクトルと類似度の高い10語とその類似度を出力せよ．

In [59]:
spain = vector("Spain")
madrid = vector("Madrid")
athens = vector("Athens")
target = spain - madrid + athens

In [60]:
print(target[:2])
top10(target)

[ 144.83628899 -119.6647485 ]
0.9340990659341353 Spain
0.7857701837877478 Italy
0.7754966286733158 Scotland
0.7718133074945547 successfully
0.7701343322332543 committed
0.7687767661597124 shared
0.7675269904616108 Russia
0.7633115748903724 enemy
0.761267222874195 attempted
0.75483174518853 faced
